In [1]:
using CSV, DataFrames, LightGraphs, SimpleWeightedGraphs, GraphPlot, UnicodePlots, Cairo, Compose, Colors, Missings, Metis;

┌ Info: Precompiling GraphPlot [a2cc645c-3eea-5389-862e-a155d0052231]
└ @ Base loading.jl:1278
┌ Info: Precompiling Cairo [159f3aea-2a34-519c-b102-8c37f9878175]
└ @ Base loading.jl:1278
┌ Info: Precompiling Metis [2679e427-3c69-5b7f-982b-ece356f1e94b]
└ @ Base loading.jl:1278


### Get the Connection Data

In [2]:
down_csv = "LC14 downstream Connections.csv";
up_csv = "LC14 Upstream Connections.csv";

down_df = DataFrame(CSV.File(down_csv));
up_df = DataFrame(CSV.File(up_csv));

# Remove the first row as it's just the numbers. 
down_df = down_df[:,2:end];
up_df = up_df[:,2:end];

##### Clean up the Missing Data

In [3]:
replace!(up_df["type_pre"], missing => "")
replace!(up_df["type_post"], missing => "")
replace!(down_df["type_pre"], missing => "")
replace!(down_df["type_post"], missing => "");

### Convert from the dataframes into a weighted graph network.

In [4]:
unique_ids = unique([up_df["bodyId_pre"]; up_df["bodyId_post"]]);
id_numbers = Dict{Int, Int}();
sizehint!(id_numbers, length(unique_ids));
for (i, id) in enumerate(unique_ids)
    id_numbers[id] = i
end

df_ids = Dict{Int, Int}();
sizehint!(df_ids, length(unique_ids));
for (i, id) in enumerate(unique_ids)
    df_ids[i] = id
end

In [5]:
g = SimpleGraph(length(unique_ids));
for row in eachrow(up_df)
    add_edge!(g, id_numbers[row["bodyId_pre"]], id_numbers[row["bodyId_post"]])
end

In [6]:
di_g = DiGraph(length(unique_ids));
for row in eachrow(up_df)
    add_edge!(di_g, id_numbers[row["bodyId_pre"]], id_numbers[row["bodyId_post"]])
end

In [7]:
weighted_g = SimpleWeightedGraph(length(unique_ids));
for row in eachrow(up_df)
    add_edge!(weighted_g, id_numbers[row["bodyId_pre"]], id_numbers[row["bodyId_post"]], row["weight"])
end

In [8]:
weighted_di_g = SimpleWeightedDiGraph(length(unique_ids));
for row in eachrow(up_df)
    add_edge!(weighted_di_g, id_numbers[row["bodyId_pre"]], id_numbers[row["bodyId_post"]], row["weight"])
end

### Color the LC14 neurons different to the rest.

In [9]:
node_colors = Vector{RGB}(undef, length(unique_ids))
lc14_color = RGB(0/255, 188/255, 255/255)
other_color = RGB(161/255, 161/255, 161/255)

for row in eachrow(up_df)
    if ismissing(row["type_pre"]) || row["type_pre"] != "LC14"
        node_colors[id_numbers[row["bodyId_pre"]]] = other_color
    else
        node_colors[id_numbers[row["bodyId_pre"]]] = lc14_color
    end
    
    if ismissing(row["type_post"]) || row["type_post"] != "LC14"
        node_colors[id_numbers[row["bodyId_post"]]] = other_color
    else
        node_colors[id_numbers[row["bodyId_post"]]] = lc14_color
    end
end
nodesize = [3 * LightGraphs.outdegree(weighted_g, v) for v in LightGraphs.vertices(weighted_g)];

In [10]:
plt = gplot(weighted_g, nodefillc=node_colors, nodesize=100 .* nodesize)
draw(PDF("upstream_colored_nodesize2.pdf", 30cm, 30cm), plt)

### Find innervation Types

In [11]:
using Plots
unicodeplots()
presynaptic_types = Vector{String}(up_df["type_pre"])
presynaptic_uniquetypes = unique(presynaptic_types)
filter!(!isempty, presynaptic_uniquetypes)

counts = Vector{Int}(undef, length(presynaptic_uniquetypes))

i = 1
for unique_type in presynaptic_uniquetypes
    counts[i] = count(unique_type .== presynaptic_types)
    i += 1
end

thresh_types = presynaptic_uniquetypes[counts .> 20]
f = bar(thresh_types, counts[counts.>20])


┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278


Plot{Plots.UnicodePlotsBackend() n=1}

## Bisection

### Some functions for finding the strongest n neighbours.

In [12]:
function strongest_neigbours(graph, vertex, n)
    
    neighbs = neighbors(graph, vertex)
    weights = neighborhood_weights(graph, vertex, neighbs)
    neighbs[strongest_weights(weights, n)]
end

function neighborhood_weights(graph, origin, neighbrs)

    i = 1
    weights = Vector{Float64}(undef, length(neighbrs))
    for n in neighbrs
        weights[i] = graph.weights[origin, n]
        i += 1
    end
    return weights
end


function strongest_weights(weights,n)

    num_weights = length(weights)
    partialsortperm(weights,1:min(num_weights,n),rev=true)
end;

In [13]:
i = 1
n = 2
graph = weighted_g

neighborhoods = Vector{Vector{Int}}(undef, nv(graph))

for vertex in 1:nv(graph)
    neighborhoods[i] = strongest_neigbours(graph, vertex, n)
    i += 1
end

# Any nodes that were strongly connected previously are now considered the same node.


Partition the graph into 2 groups

In [14]:
node_groups = Metis.partition(g, 2)
i=1
node_colors = Vector{RGB}(undef, length(unique_ids))
for node in node_groups
    node_colors[i] = (node_groups[i] == 1) ? RGB(255/255, 188/255, 0/255) : RGB(0/255, 188/255, 255/255)
    i += 1
end;

In [22]:
plt = gplot(weighted_g, nodefillc=node_colors)
plt;

In [68]:
RGB(255/255, 188/255, 0/255)

In [16]:
up_df.pre_group = zeros(Int, size(up_df,1))
up_df.post_group = zeros(Int, size(up_df,1))

for row in eachrow(up_df)
    
    row["pre_group"] = node_groups[id_numbers[row["bodyId_pre"]]]
    row["post_group"] = node_groups[id_numbers[row["bodyId_post"]]]

end

In [17]:
group_ids = DataFrame()

out = Array{Int,2}(undef, length(unique_ids), 2);

for (i,node) in enumerate(node_groups)
    
    out[i,:] .= [df_ids[i], node]
end

upstream_grouped = DataFrame()
upstream_grouped.bodyIds = out[:,1]
upstream_grouped.group = out[:,2];
CSV.write("upstream neuron groups.csv", upstream_grouped)

"upstream neuron groups.csv"

In [67]:
CSV.write("upstream neuron groups (with metadata).csv", up_df)

"upstream neuron groups (with metadata).csv"